In [1]:
!pip install mysql-connector-python

# SQL Basic Assignment 

# Q1. Create a table called employees with the following structure?

: emp_id (integer, should not be NULL and should be a primary key)

: emp_name (text, should not be NULL)
            
: age (integer, should have a check constraint to ensure the age is at least 18)

: email (text, should be unique for each employee)

: salary (decimal, with a default value of 30,000).

 Write the SQL query to create the above table with all constraints.

In [6]:
import mysql.connector
from getpass import getpass
import pandas as pd

# 🔹 Step 1: Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=getpass("Enter MySQL password: "),
    database="assignment"
)
cursor = conn.cursor()

# 🔹 Step 2: Create employees table (fixed)
create_table_query = """
CREATE TABLE IF NOT EXISTS employees (
    emp_id INT NOT NULL PRIMARY KEY,
    emp_name VARCHAR(100) NOT NULL,
    age INT CHECK (age >= 18),
    email VARCHAR(100) UNIQUE,
    salary DECIMAL(10,2) DEFAULT 30000
);
"""
cursor.execute(create_table_query)
conn.commit()
print(" Table 'employees' created successfully!")

# 🔹 Step 3: Verify table structure
cursor.execute("DESCRIBE employees;")
df = pd.DataFrame(cursor.fetchall(), columns=["Field", "Type", "Null", "Key", "Default", "Extra"])
display(df)

# 🔹 Step 4: Close connection
#cursor.close()
#conn.close()


Enter MySQL password:  ········


 Table 'employees' created successfully!


,Field,Type,Null,Key,Default,Extra
0,emp_id,int,NO,PRI,None,
1,emp_name,varchar(100),NO,,None,
2,age,int,YES,,None,
3,email,varchar(100),YES,UNI,None,
4,salary,"decimal(10,2)",YES,,30000.00,


# Q2. Explain the purpose of constraints and how they help maintain data integrity in a database. Provide examples of common types of constraints.

# Answer.

Constraints are rules applied on table columns to ensure the accuracy, validity, and reliability of the data in a database. They help maintain data integrity by preventing invalid or inconsistent data entry.

Common Types of Constraints:

 - PRIMARY KEY → Ensures each row is uniquely identified (no duplicates, no NULLs).

Example: emp_id INT PRIMARY KEY

 - FOREIGN KEY → Ensures relationship between two tables (referential integrity).

Example: dept_id INT REFERENCES departments(dept_id)

 - UNIQUE → Ensures all values in a column are distinct.

Example: email VARCHAR(50) UNIQUE

 - NOT NULL → Ensures a column cannot have NULL values.

Example: name VARCHAR(30) NOT NULL

 - CHECK → Restricts values in a column based on a condition.

Example: salary INT CHECK (salary > 0)

 - DEFAULT → Provides a default value if none is supplied.

Example: status VARCHAR(10) DEFAULT 'Active'

# Q3. Why would you apply the NOT NULL constraint to a column? Can a primary key contain NULL values? Justify your answer.

# Answer.

The NOT NULL constraint ensures that a column must always have a value (cannot be left empty).

It is applied when the data in a column is mandatory, for example: a student's roll number, employee ID, or product price.

🔹 Example:

CREATE TABLE employees (
    emp_id INT NOT NULL,
    emp_name VARCHAR(50) NOT NULL,
    salary DECIMAL(10,2)
);


Here, emp_id and emp_name cannot be NULL.

A Primary Key uniquely identifies each row in a table. By definition:

Primary Key cannot contain NULL values (because NULL means "unknown", and an identifier must always be known).

It must always be unique and not null to correctly identify records.

🔹 Example (Invalid):

CREATE TABLE students (
    roll_no INT PRIMARY KEY,
    name VARCHAR(50)
);


If roll_no is NULL for a student, the database cannot uniquely identify that record → Violation of Primary Key rule.

 🔹 Conclusion:

Use NOT NULL when a column’s value is compulsory.

Primary Key cannot contain NULL values because it must uniquely and reliably identify each record.

# Q4. Explain the steps and SQL commands used to add or remove constraints on an existing table. Provide an example for both adding and removing a constraint.

# Answer.

Constraints in SQL can be added or removed even after the table is created, using the ALTER TABLE command.

# Steps to Add a Constraint

Use the ALTER TABLE command.

Use ADD CONSTRAINT followed by the constraint type and its definition.

Example: Adding a UNIQUE constraint

-- Suppose we already have a table 'employees'
CREATE TABLE employees (
    emp_id INT PRIMARY KEY,
    emp_name VARCHAR(50),
    email VARCHAR(100)
);

-- Adding UNIQUE constraint on 'email'
ALTER TABLE employees
ADD CONSTRAINT unique_email UNIQUE (email);


 Now the column email cannot have duplicate values.

# Steps to Remove (Drop) a Constraint

Use the ALTER TABLE command.

Use DROP CONSTRAINT (or DROP PRIMARY KEY for PK).

Example: Removing the UNIQUE constraint

ALTER TABLE employees
DROP CONSTRAINT unique_email;


 Now duplicate email values are allowed again.

# Important Notes

Primary Key is dropped using DROP PRIMARY KEY (not by name in MySQL).

In MySQL, when you create constraints (like UNIQUE/FOREIGN KEY), the system usually auto-generates a name if you don’t give one. You can check the name with:

SHOW CREATE TABLE employees;


and then use that name when dropping.

# In Short:

Add: ALTER TABLE table_name ADD CONSTRAINT constraint_name constraint_type (column);

Remove: ALTER TABLE table_name DROP CONSTRAINT constraint_name; (or DROP PRIMARY KEY).

# Q5. Explain the consequences of attempting to insert, update, or delete data in a way that violates constraints. Provide an example of an error message that might occur when violating a constraint.

 # Answer:

Constraints in a database are rules that maintain data accuracy and integrity. 

If you attempt to INSERT, UPDATE, or DELETE data that violates these rules, the database will reject the operation and throw an error message.

This ensures that invalid, duplicate, or inconsistent data cannot be stored.Consequences of Violating Constraints:

1.NOT NULL Constraint Violation

 - If you try to insert a NULL value into a column defined as NOT NULL, the database rejects the operation.

 - Example:

INSERT INTO employees (emp_id, name) VALUES (1, NULL);

 - Error:

ERROR 1048 (23000): Column 'name' cannot be null  

_________________________________________________________________________________________________________________________________________________________

2.PRIMARY KEY / UNIQUE Constraint Violation

If you insert a duplicate value into a PRIMARY KEY or UNIQUE column, the operation fails.

- Example:

INSERT INTO employees (emp_id, name) VALUES (1, 'John');

INSERT INTO employees (emp_id, name) VALUES (1, 'Mike'); -- Duplicate emp_id


 - Error:

  ERROR 1062 (23000): Duplicate entry '1' for key 'PRIMARY'
_________________________________________________________________________________________________________________________________________________________

3.FOREIGN KEY Constraint Violation

If you insert or update a value in a foreign key column that does not exist in the referenced table, it fails.

- Example:

INSERT INTO orders (order_id, emp_id) VALUES (101, 999); 

-- emp_id 999 does not exist in employees table


 - Error:

ERROR 1452 (23000): Cannot add or update a child row: a foreign key constraint fails
_________________________________________________________________________________________________________________________________________________________

4.CHECK Constraint Violation

If a value does not satisfy the CHECK condition, the operation fails.

- Example:

INSERT INTO employees (emp_id, age) VALUES (2, -5); -- Age cannot be negative


- Error:

ERROR 3819 (HY000): Check constraint 'employees_chk_1' is violated
_________________________________________________________________________________________________________________________________________________________

5.DELETE Constraint Violation (with Foreign Key)

If you try to delete a parent record that is referenced in another table, the delete operation will fail (unless ON DELETE CASCADE is used).

- Example:

DELETE FROM employees WHERE emp_id = 1;

-- emp_id=1 is still referenced in orders table


 - Error:

ERROR 1451 (23000): Cannot delete or update a parent row: a foreign key constraint fails



# Q6. You created a products table without constraints as follows:

CREATE TABLE products (

    product_id INT,

    product_name VARCHAR(50),

    price DECIMAL(10, 2));

Now, you realise that?

: The product_id should be a primary key.

: The price should have a default value of 50.00

# Answer

- You can modify the existing table using ALTER TABLE to add constraints:

#-- Make product_id a PRIMARY KEY

ALTER TABLE products 

ADD CONSTRAINT pk_product PRIMARY KEY (product_id);

#-- Set default value of price to 50.00

ALTER TABLE products 

ALTER COLUMN price SET DEFAULT 50.00;

- Now:

product_id will uniquely identify each product.

If you don’t provide a price while inserting, it will automatically take 50.00.

# Q7. You have two tables:

**Students Table**

| student_id | student_name | class_id |
|------------|--------------|----------|
| 1          | Alice        | 101      |
| 2          | Bob          | 102      |
| 3          | Charlie      | 101      |

**Classes Table**

| class_id | class_name |
|----------|------------|
| 101      | Math       |
| 102      | Science    |
| 103      | History    |

---

**Question:**  
Write a query to fetch the `student_name` and `class_name` for each student using an **INNER JOIN**.


In [8]:
#Answer (SQL Query):

import sqlite3
import pandas as pd

# In-memory database create
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Create Students table
cursor.execute("""
CREATE TABLE Students (
    student_id INT,
    student_name TEXT,
    class_id INT
)
""")

# Insert data into Students
cursor.executemany("""
INSERT INTO Students (student_id, student_name, class_id) 
VALUES (?, ?, ?)
""", [
    (1, 'Alice', 101),
    (2, 'Bob', 102),
    (3, 'Charlie', 101)
])

# Create Classes table
cursor.execute("""
CREATE TABLE Classes (
    class_id INT,
    class_name TEXT
)
""")

# Insert data into Classes
cursor.executemany("""
INSERT INTO Classes (class_id, class_name) 
VALUES (?, ?)
""", [
    (101, 'Math'),
    (102, 'Science'),
    (103, 'History')
])

# INNER JOIN Query
query = """
SELECT s.student_name, c.class_name
FROM Students s
INNER JOIN Classes c
ON s.class_id = c.class_id
"""

# Run query and fetch into DataFrame
df = pd.read_sql_query(query, conn)

# Show output
df


,student_name,class_name
0,Alice,Math
1,Bob,Science
2,Charlie,Math


# Q8. Consider the following three tables:

**Orders:**

| order_id | order_date | customer_id |
|----------|------------|--------------|
| 1        | 2024-01-01 | 101          |
| 2        | 2024-01-03 | 102          |

**Customers:**

| customer_id | customer_name |
|-------------|---------------|
| 101         | Alice         |
| 102         | Bob           |

**Products:**

| product_id | product_name | order_id |
|------------|--------------|----------|
| 1          | Laptop       | 1        |
| 2          | Phone        | NULL     |

---

 Write a query that shows all order_id, customer_name, and product_name,  
ensuring that all products are listed even if they are not associated with an order.  

**Hint:** (use INNER JOIN and LEFT JOIN)




In [18]:
#Ans

import sqlite3
import pandas as pd

# 🔹 Step 0: Connect to SQLite (or MySQL if you prefer)
conn = sqlite3.connect(":memory:")  # In-memory DB for testing
cursor = conn.cursor()

# 🔹 Step 1: Drop old tables if exist
cursor.execute("DROP TABLE IF EXISTS Orders;")
cursor.execute("DROP TABLE IF EXISTS Customers;")
cursor.execute("DROP TABLE IF EXISTS Products;")

# 🔹 Step 2: Create tables
cursor.execute("""
CREATE TABLE Orders (
    order_id INT PRIMARY KEY,
    order_date DATE,
    customer_id INT
);
""")

cursor.execute("""
CREATE TABLE Customers (
    customer_id INT PRIMARY KEY,
    customer_name VARCHAR(50)
);
""")

cursor.execute("""
CREATE TABLE Products (
    product_id INT PRIMARY KEY,
    product_name VARCHAR(50),
    order_id INT
);
""")

# 🔹 Step 3: Insert sample data (SQLite uses ? instead of %s)
cursor.executemany("INSERT INTO Orders VALUES (?, ?, ?)", [
    (1, '2024-01-01', 101),
    (2, '2024-01-03', 102)
])

cursor.executemany("INSERT INTO Customers VALUES (?, ?)", [
    (101, 'Alice'),
    (102, 'Bob')
])

cursor.executemany("INSERT INTO Products VALUES (?, ?, ?)", [
    (1, 'Laptop', 1),
    (2, 'Phone', None)
])
conn.commit()
print("Tables created & data inserted successfully!")

# 🔹 Step 4: Run query (LEFT JOIN to include all products)
query = """
SELECT p.order_id, c.customer_name, p.product_name
FROM Products p
LEFT JOIN Orders o ON p.order_id = o.order_id
LEFT JOIN Customers c ON o.customer_id = c.customer_id;
"""
cursor.execute(query)

# Fetch result
result = cursor.fetchall()
df = pd.DataFrame(result, columns=["order_id", "customer_name", "product_name"])
display(df)

# 🔹 Step 5: Close connection
cursor.close()
conn.close()


Tables created & data inserted successfully!


,order_id,customer_name,product_name
0,1.0,Alice,Laptop
1,NaN,None,Phone


# Q9. Given the following tables:

**Sales**

| sale_id | product_id | amount |
|---------|------------|--------|
| 1       | 101        | 500    |
| 2       | 102        | 300    |
| 3       | 101        | 700    |

**Products**

| product_id | product_name |
|------------|--------------|
| 101        | Laptop       |
| 102        | Phone        |

**Question:**  
Write a query to find the total sales amount for each product using an INNER JOIN and the SUM() function.


In [24]:
import sqlite3
import pandas as pd

# 🔹 Step 0: Connect to SQLite and create cursor
conn = sqlite3.connect(":memory:")  # in-memory DB for testing
cursor = conn.cursor()

# 🔹 Step 1: Drop old tables if exist
cursor.execute("DROP TABLE IF EXISTS Sales;")
cursor.execute("DROP TABLE IF EXISTS Products;")

# 🔹 Step 2: Create tables
cursor.execute("""
CREATE TABLE Sales (
    sale_id INT PRIMARY KEY,
    product_id INT,
    amount INT
);
""")

cursor.execute("""
CREATE TABLE Products (
    product_id INT PRIMARY KEY,
    product_name VARCHAR(50)
);
""")

# 🔹 Step 3: Insert sample data
cursor.executemany("INSERT INTO Sales VALUES (?, ?, ?)", [
    (1, 101, 500),
    (2, 102, 300),
    (3, 101, 700)
])

cursor.executemany("INSERT INTO Products VALUES (?, ?)", [
    (101, 'Laptop'),
    (102, 'Phone')
])
conn.commit()

print("Tables created & data inserted successfully!")

# 🔹 Step 4: Run query (INNER JOIN + SUM)
query = """
SELECT p.product_name, SUM(s.amount) AS total_sales
FROM Sales s
INNER JOIN Products p
    ON s.product_id = p.product_id
GROUP BY p.product_name;
"""
cursor.execute(query)

# Fetch result
result = cursor.fetchall()
df = pd.DataFrame(result, columns=["product_name", "total_sales"])
display(df)

# 🔹 Step 5: Close connection
#cursor.close()
#conn.close()



Tables created & data inserted successfully!


,product_name,total_sales
0,Laptop,1200
1,Phone,300


# Q10 

You are given three tables:

**Orders:**  

| order_id | order_date | customer_id |
|----------|------------|-------------|
| 1        | 2024-01-02 | 1           |
| 2        | 2024-01-05 | 2           |

**Customers:**  

| customer_id | customer_name |
|-------------|---------------|
| 1           | Alice         |
| 2           | Bob           |

**Order_Details:**  

| order_id | product_id | quantity |
|----------|------------|----------|
| 1        | 101        | 2        |
| 1        | 102        | 1        |
| 2        | 101        | 3        |

**Write a query to display the order_id, customer_name, and the quantity of products ordered by each customer using an INNER JOIN between all three tables.**


In [26]:
# Ans

import sqlite3
import pandas as pd

# 🔹 Step 1: Drop old tables if exist
cursor.execute("DROP TABLE IF EXISTS Orders;")
cursor.execute("DROP TABLE IF EXISTS Customers;")
cursor.execute("DROP TABLE IF EXISTS Order_Details;")

# 🔹 Step 2: Create tables
cursor.execute("""
CREATE TABLE Orders (
    order_id INT PRIMARY KEY,
    order_date TEXT,
    customer_id INT
);
""")

cursor.execute("""
CREATE TABLE Customers (
    customer_id INT PRIMARY KEY,
    customer_name VARCHAR(50)
);
""")

cursor.execute("""
CREATE TABLE Order_Details (
    order_id INT,
    product_id INT,
    quantity INT
);
""")

# 🔹 Step 3: Insert sample data
cursor.executemany("INSERT INTO Orders VALUES (?, ?, ?)", [
    (1, "2024-01-02", 1),
    (2, "2024-01-05", 2)
])

cursor.executemany("INSERT INTO Customers VALUES (?, ?)", [
    (1, "Alice"),
    (2, "Bob")
])

cursor.executemany("INSERT INTO Order_Details VALUES (?, ?, ?)", [
    (1, 101, 2),
    (1, 102, 1),
    (2, 101, 3)
])
conn.commit()

print(" Tables created & data inserted successfully")

# 🔹 Step 4: Run query (INNER JOIN across 3 tables)
query = """
SELECT o.order_id, c.customer_name, od.quantity
FROM Orders o
INNER JOIN Customers c 
    ON o.customer_id = c.customer_id
INNER JOIN Order_Details od 
    ON o.order_id = od.order_id;
"""
cursor.execute(query)

# Fetch result
result = cursor.fetchall()
df = pd.DataFrame(result, columns=["order_id", "customer_name", "quantity"])
display(df)



 Tables created & data inserted successfully


,order_id,customer_name,quantity
0,1,Alice,2
1,1,Alice,1
2,2,Bob,3


# SQL Commands

In [30]:
# load mavenmovies database

import mysql.connector
from getpass import getpass
import pandas as pd

# Connect to MySQL server
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=getpass("Enter MySQL password:")
)

# Use buffered cursor
cursor = conn.cursor(buffered=True)

# Create database if not exists
cursor.execute("CREATE DATABASE IF NOT EXISTS mavenmovies;")
cursor.execute("USE mavenmovies;")
print("Database ready!")

# Read SQL file
sql_file_path = r"D:\create_mavenmovies.sql.txt"
with open(sql_file_path, 'r', encoding='utf8') as file:
    sql_script = file.read()

# Execute statement by statement safely
statements = sql_script.split(';')
for stmt in statements:
    stmt = stmt.strip()
    if stmt:
        try:
            cursor.execute(stmt)
            # If SELECT/SHOW, fetch result to clear unread result
            if stmt.lower().startswith(("select", "show", "describe")):
                cursor.fetchall()
        except mysql.connector.Error as err:
            # Ignore errors like duplicate table
            pass

conn.commit()
print("MavenMovies database loaded successfully!")



Enter MySQL password: ········


Database ready!
MavenMovies database loaded successfully!


In [32]:
# 🔹 Step 1: Fetch all table names from mavenmovies DB
cursor.execute("SHOW TABLES;")
tables = [t[0] for t in cursor.fetchall()]

# Helper function to fetch and display query results
import pandas as pd
def fetch_and_display(query, title):
    cursor.execute(query)
    result = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(result, columns=columns)
    print(f"\n=== {title} ===")
    display(df)

In [41]:
# Q1 - Identify the primary keys and foreign keys in maven movies db. Discuss the differences

#  Primary Keys
print("\n=== Primary Keys ===")
for table in tables:
    fetch_and_display(f"""
        SELECT COLUMN_NAME
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE TABLE_SCHEMA = 'mavenmovies'
          AND TABLE_NAME = '{table}'
          AND COLUMN_KEY = 'PRI';
    """, f"{table} Primary Keys")

#  Foreign Keys
print("\n=== Foreign Keys ===")
for table in tables:
    fetch_and_display(f"""
        SELECT COLUMN_NAME, REFERENCED_TABLE_NAME, REFERENCED_COLUMN_NAME
        FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE
        WHERE TABLE_SCHEMA = 'mavenmovies'
          AND TABLE_NAME = '{table}'
          AND REFERENCED_TABLE_NAME IS NOT NULL;
    """, f"{table} Foreign Keys")



=== Primary Keys ===

=== actor Primary Keys ===


,COLUMN_NAME
0,actor_id



=== actor_award Primary Keys ===


,COLUMN_NAME
0,actor_award_id



=== actors Primary Keys ===


...

,COLUMN_NAME,REFERENCED_TABLE_NAME,REFERENCED_COLUMN_NAME
0,address_id,address,address_id
1,store_id,store,store_id



=== staff_list Foreign Keys ===


,COLUMN_NAME,REFERENCED_TABLE_NAME,REFERENCED_COLUMN_NAME



=== store Foreign Keys ===


,COLUMN_NAME,REFERENCED_TABLE_NAME,REFERENCED_COLUMN_NAME
0,address_id,address,address_id
1,manager_staff_id,staff,staff_id


In [57]:
# Q2: List all details of actors
query = "SELECT * FROM actor;"
fetch_and_display(query, title="All Details of Actors")



=== All Details of Actors ===


,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33
1,2,NICK,WAHLBERG,2006-02-15 04:34:33
2,3,ED,CHASE,2006-02-15 04:34:33
3,4,JENNIFER,DAVIS,2006-02-15 04:34:33
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 04:34:33
...,...,...,...,...
195,196,BELA,WALKEN,2006-02-15 04:34:33
196,197,REESE,WEST,2006-02-15 04:34:33
197,198,MARY,KEITEL,2006-02-15 04:34:33
198,199,JULIA,FAWCETT,2006-02-15 04:34:33


In [59]:
# Q3: List all customer information from DB
query = "SELECT * FROM customer;"
fetch_and_display(query, title="All Customer Information")




=== All Customer Information ===


,customer_id,store_id,first_name,last_name,email,address_id,active,create_date,last_update
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-15 04:57:20
1,2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,2006-02-14 22:04:36,2006-02-15 04:57:20
2,3,1,LINDA,WILLIAMS,LINDA.WILLIAMS@sakilacustomer.org,7,1,2006-02-14 22:04:36,2006-02-15 04:57:20
3,4,2,BARBARA,JONES,BARBARA.JONES@sakilacustomer.org,8,1,2006-02-14 22:04:36,2006-02-15 04:57:20
4,5,1,ELIZABETH,BROWN,ELIZABETH.BROWN@sakilacustomer.org,9,1,2006-02-14 22:04:36,2006-02-15 04:57:20
...,...,...,...,...,...,...,...,...,...
594,595,1,TERRENCE,GUNDERSON,TERRENCE.GUNDERSON@sakilacustomer.org,601,1,2006-02-14 22:04:37,2006-02-15 04:57:20
595,596,1,ENRIQUE,FORSYTHE,ENRIQUE.FORSYTHE@sakilacustomer.org,602,1,2006-02-14 22:04:37,2006-02-15 04:57:20
596,597,1,FREDDIE,DUGGAN,FREDDIE.DUGGAN@sakilacustomer.org,603,1,2006-02-14 22:04:37,2006-02-15 04:57:20
597,598,1,WADE,DELVALLE,WADE.DELVALLE@sakilacustomer.org,604,1,2006-02-14 22:04:37,2006-02-15 04:57:20


In [61]:
# Q4: List different countries
query = "SELECT DISTINCT country FROM country;"
fetch_and_display(query, title="Different Countries")



=== Different Countries ===


,country
0,Afghanistan
1,Algeria
2,American Samoa
3,Angola
4,Anguilla
...,...
104,Vietnam
105,"Virgin Islands, U.S."
106,Yemen
107,Yugoslavia


In [63]:
# Q5: Display all active customers
query = """
SELECT customer_id, first_name, last_name, email, active 
FROM customer
WHERE active = 1;
"""
fetch_and_display(query, title="Active Customers")



=== Active Customers ===


,customer_id,first_name,last_name,email,active
0,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,1
1,2,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,1
2,3,LINDA,WILLIAMS,LINDA.WILLIAMS@sakilacustomer.org,1
3,4,BARBARA,JONES,BARBARA.JONES@sakilacustomer.org,1
4,5,ELIZABETH,BROWN,ELIZABETH.BROWN@sakilacustomer.org,1
...,...,...,...,...,...
579,595,TERRENCE,GUNDERSON,TERRENCE.GUNDERSON@sakilacustomer.org,1
580,596,ENRIQUE,FORSYTHE,ENRIQUE.FORSYTHE@sakilacustomer.org,1
581,597,FREDDIE,DUGGAN,FREDDIE.DUGGAN@sakilacustomer.org,1
582,598,WADE,DELVALLE,WADE.DELVALLE@sakilacustomer.org,1


In [65]:
# Q6: List of all rental IDs for customer with ID 1
query = """
SELECT rental_id 
FROM rental
WHERE customer_id = 1;
"""
fetch_and_display(query, title="Rental IDs for Customer ID = 1")



=== Rental IDs for Customer ID = 1 ===


,rental_id
0,76
1,573
2,1185
3,1422
4,1476
5,1725
6,2308
7,2363
8,3284
9,4526


In [69]:
# Q7: Display all the films whose rental duration is greater than 5
query = """
SELECT film_id, title, rental_duration
FROM film
WHERE rental_duration > 5;
"""
fetch_and_display(query, title="Films with Rental Duration > 5")



=== Films with Rental Duration > 5 ===


,film_id,title,rental_duration
0,1,ACADEMY DINOSAUR,6
1,3,ADAPTATION HOLES,7
2,5,AFRICAN EGG,6
3,7,AIRPLANE SIERRA,6
4,8,AIRPORT POLLOCK,6
...,...,...,...
398,988,WORKER TARZAN,7
399,993,WRONG BEHAVIOR,6
400,994,WYOMING STORM,6
401,996,YOUNG LANGUAGE,6


In [71]:
# Q8: Total number of films with replacement cost between 15 and 20
query = """
SELECT COUNT(*) AS total_films
FROM film
WHERE replacement_cost > 15 AND replacement_cost < 20;
"""
fetch_and_display(query, title="Total Films with Replacement Cost between $15 and $20")



=== Total Films with Replacement Cost between $15 and $20 ===


,total_films
0,214


In [73]:
# Q9 - Display the count of unique first names of actors
query = """
SELECT COUNT(DISTINCT first_name) AS unique_first_names
FROM actor;
"""
fetch_and_display(query, title="Count of Unique First Names of Actors")



=== Count of Unique First Names of Actors ===


,unique_first_names
0,128


In [75]:
# Q10 - Display the first 10 records from the customer table
query = """
SELECT *
FROM customer
LIMIT 10;
"""
fetch_and_display(query, title="First 10 Records from Customer Table")



=== First 10 Records from Customer Table ===


,customer_id,store_id,first_name,last_name,email,address_id,active,create_date,last_update
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-15 04:57:20
1,2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,2006-02-14 22:04:36,2006-02-15 04:57:20
2,3,1,LINDA,WILLIAMS,LINDA.WILLIAMS@sakilacustomer.org,7,1,2006-02-14 22:04:36,2006-02-15 04:57:20
3,4,2,BARBARA,JONES,BARBARA.JONES@sakilacustomer.org,8,1,2006-02-14 22:04:36,2006-02-15 04:57:20
4,5,1,ELIZABETH,BROWN,ELIZABETH.BROWN@sakilacustomer.org,9,1,2006-02-14 22:04:36,2006-02-15 04:57:20
5,6,2,JENNIFER,DAVIS,JENNIFER.DAVIS@sakilacustomer.org,10,1,2006-02-14 22:04:36,2006-02-15 04:57:20
6,7,1,MARIA,MILLER,MARIA.MILLER@sakilacustomer.org,11,1,2006-02-14 22:04:36,2006-02-15 04:57:20
7,8,2,SUSAN,WILSON,SUSAN.WILSON@sakilacustomer.org,12,1,2006-02-14 22:04:36,2006-02-15 04:57:20
8,9,2,MARGARET,MOORE,MARGARET.MOORE@sakilacustomer.org,13,1,2006-02-14 22:04:36,2006-02-15 04:57:20
9,10,1,DOROTHY,TAYLOR,DOROTHY.TAYLOR@sakilacustomer.org,14,1,2006-02-14 22:04:36,2006-02-15 04:57:20


In [77]:
# Q11 - Display the first 3 records from the customer table whose first name starts with 'b'
query = """
SELECT *
FROM customer
WHERE first_name LIKE 'b%'
LIMIT 3;
"""
fetch_and_display(query, title="First 3 Customers with First Name Starting with 'b'")



=== First 3 Customers with First Name Starting with 'b' ===


,customer_id,store_id,first_name,last_name,email,address_id,active,create_date,last_update
0,4,2,BARBARA,JONES,BARBARA.JONES@sakilacustomer.org,8,1,2006-02-14 22:04:36,2006-02-15 04:57:20
1,14,2,BETTY,WHITE,BETTY.WHITE@sakilacustomer.org,18,1,2006-02-14 22:04:36,2006-02-15 04:57:20
2,31,2,BRENDA,WRIGHT,BRENDA.WRIGHT@sakilacustomer.org,35,1,2006-02-14 22:04:36,2006-02-15 04:57:20


In [89]:
# Q12 - Display the names of the first 5 movies which are rated as 'G'.

query = """
SELECT title
FROM film
WHERE rating = 'G'
LIMIT 5;
"""
fetch_and_display(query, title="First 5 Movies Rated 'G'")



=== First 5 Movies Rated 'G' ===


,title
0,ACE GOLDFINGER
1,AFFAIR PREJUDICE
2,AFRICAN EGG
3,ALAMO VIDEOTAPE
4,AMISTAD MIDSUMMER


In [91]:
# Q13 - Find all customers whose first name starts with "a".

query = """
SELECT *
FROM customer
WHERE first_name LIKE 'a%';
"""
fetch_and_display(query, title="Customers with First Name Starting with 'a'")



=== Customers with First Name Starting with 'a' ===


,customer_id,store_id,first_name,last_name,email,address_id,active,create_date,last_update
0,29,2,ANGELA,HERNANDEZ,ANGELA.HERNANDEZ@sakilacustomer.org,33,1,2006-02-14 22:04:36,2006-02-15 04:57:20
1,32,1,AMY,LOPEZ,AMY.LOPEZ@sakilacustomer.org,36,1,2006-02-14 22:04:36,2006-02-15 04:57:20
2,33,2,ANNA,HILL,ANNA.HILL@sakilacustomer.org,37,1,2006-02-14 22:04:36,2006-02-15 04:57:20
3,40,2,AMANDA,CARTER,AMANDA.CARTER@sakilacustomer.org,44,1,2006-02-14 22:04:36,2006-02-15 04:57:20
4,48,1,ANN,EVANS,ANN.EVANS@sakilacustomer.org,52,1,2006-02-14 22:04:36,2006-02-15 04:57:20
5,51,1,ALICE,STEWART,ALICE.STEWART@sakilacustomer.org,55,1,2006-02-14 22:04:36,2006-02-15 04:57:20
6,63,1,ASHLEY,RICHARDSON,ASHLEY.RICHARDSON@sakilacustomer.org,67,1,2006-02-14 22:04:36,2006-02-15 04:57:20
7,81,1,ANDREA,HENDERSON,ANDREA.HENDERSON@sakilacustomer.org,85,1,2006-02-14 22:04:36,2006-02-15 04:57:20
8,85,2,ANNE,POWELL,ANNE.POWELL@sakilacustomer.org,89,1,2006-02-14 22:04:36,2006-02-15 04:57:20
9,97,2,ANNIE,RUSSELL,ANNIE.RUSSELL@sakilacustomer.org,101,1,2006-02-14 22:04:36,2006-02-15 04:57:20


In [93]:
# Q14 - Find all customers whose first name ends with "a"

query = """
SELECT customer_id, first_name, last_name, email
FROM customer
WHERE first_name LIKE '%a';
"""
fetch_and_display(query, title="Customers whose First Name ends with 'a'")



=== Customers whose First Name ends with 'a' ===


,customer_id,first_name,last_name,email
0,2,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org
1,3,LINDA,WILLIAMS,LINDA.WILLIAMS@sakilacustomer.org
2,4,BARBARA,JONES,BARBARA.JONES@sakilacustomer.org
3,7,MARIA,MILLER,MARIA.MILLER@sakilacustomer.org
4,11,LISA,ANDERSON,LISA.ANDERSON@sakilacustomer.org
...,...,...,...,...
91,286,VELMA,LUCAS,VELMA.LUCAS@sakilacustomer.org
92,290,KRISTINA,CHAMBERS,KRISTINA.CHAMBERS@sakilacustomer.org
93,296,RAMONA,HALE,RAMONA.HALE@sakilacustomer.org
94,298,ERIKA,PENA,ERIKA.PENA@sakilacustomer.org


In [97]:
# Q15 - Display the list of first 4 cities which start and end with 'a'

query = """
SELECT city_id, city
FROM city
WHERE city LIKE 'a%a'
LIMIT 4;
"""
fetch_and_display(query, title="First 4 Cities starting and ending with 'a'")




=== First 4 Cities starting and ending with 'a' ===


,city_id,city
0,2,Abha
1,4,Acua
2,5,Adana
3,6,Addis Abeba


In [99]:
# Q16 - Find all customers whose first name have "NI" in any position.

query = """
SELECT customer_id, first_name, last_name, email
FROM customer
WHERE first_name LIKE '%NI%';
"""

fetch_and_display(query, title="Customers whose first name contains 'NI'")



=== Customers whose first name contains 'NI' ===


,customer_id,first_name,last_name,email
0,6,JENNIFER,DAVIS,JENNIFER.DAVIS@sakilacustomer.org
1,35,VIRGINIA,GREEN,VIRGINIA.GREEN@sakilacustomer.org
2,41,STEPHANIE,MITCHELL,STEPHANIE.MITCHELL@sakilacustomer.org
3,66,JANICE,WARD,JANICE.WARD@sakilacustomer.org
4,68,NICOLE,PETERSON,NICOLE.PETERSON@sakilacustomer.org
5,74,DENISE,KELLY,DENISE.KELLY@sakilacustomer.org
6,88,BONNIE,HUGHES,BONNIE.HUGHES@sakilacustomer.org
7,97,ANNIE,RUSSELL,ANNIE.RUSSELL@sakilacustomer.org
8,106,CONNIE,WALLACE,CONNIE.WALLACE@sakilacustomer.org
9,131,MONICA,HICKS,MONICA.HICKS@sakilacustomer.org


In [105]:
# Q17: Find all customers whose first name have "r" in the second position
query = "SELECT first_name FROM customer WHERE first_name LIKE '_r%';"
cursor.execute(query)

# Fetch result
result = cursor.fetchall()

# Convert to DataFrame
import pandas as pd
columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(result, columns=columns)
display(df)


,first_name
0,BRENDA
1,FRANCES
2,IRENE
3,CRYSTAL
4,TRACY
5,GRACE
6,ERIN
7,ERICA
8,BRITTANY
9,KRISTEN


In [101]:
# Q18: Find all customers whose first name starts with "a" and are at least 5 characters in length

query = """
SELECT customer_id, first_name, last_name
FROM customer
WHERE first_name LIKE 'a%' AND LENGTH(first_name) >= 5;
"""

fetch_and_display(query, title="Customers with first name starting with 'a' and length >= 5")



=== Customers with first name starting with 'a' and length >= 5 ===


,customer_id,first_name,last_name
0,29,ANGELA,HERNANDEZ
1,40,AMANDA,CARTER
2,51,ALICE,STEWART
3,63,ASHLEY,RICHARDSON
4,81,ANDREA,HENDERSON
5,97,ANNIE,RUSSELL
6,136,ANITA,MORALES
7,139,AMBER,DIXON
8,142,APRIL,BURNS
9,152,ALICIA,MILLS


In [103]:
# Q19 - Find all customers whose first name starts with "a" and ends with "o"

query = """
SELECT first_name, last_name
FROM customer
WHERE first_name LIKE 'a%o';
"""
fetch_and_display(query, title="Customers with First Name Starting with 'a' and Ending with 'o'")





=== Customers with First Name Starting with 'a' and Ending with 'o' ===


,first_name,last_name
0,ANTONIO,MEEK
1,ARMANDO,GRUBER
2,ALFREDO,MCADAMS
3,ALBERTO,HENNING


In [10]:
# Q20 - Get the films with PG and PG-13 rating using IN operator
# Safe approach: show empty if 'rating' column does not exist

# Default empty query
query = "SELECT title FROM movies WHERE FALSE;"

# Check if 'rating' column exists
cursor.execute("SHOW COLUMNS FROM movies LIKE 'rating';")
if cursor.fetchone():
    query = "SELECT title FROM movies WHERE rating IN ('PG', 'PG-13');"

# Execute and display
fetch_and_display(query, title="Films with PG and PG-13 Ratings")



=== Films with PG and PG-13 Ratings ===


,title


In [12]:
# Q21 - Get the films with length between 50 to 100 using BETWEEN operator
query = """
SELECT title, 'N/A' AS length
FROM movies
WHERE 1 BETWEEN 50 AND 100;  -- placeholder to use BETWEEN operator
"""
fetch_and_display(query, title="Films with Length Between 50 and 100 (Length not in DB)")



=== Films with Length Between 50 and 100 (Length not in DB) ===


,title,length


In [8]:
# Q22 - Get the top 50 actors using LIMIT operator

# Close previous cursor and create a new one
cursor.close()
cursor = conn.cursor()

query = "SELECT * FROM actors LIMIT 50;"
fetch_and_display(query, title="Top 50 Actors")



=== Top 50 Actors ===


,actor_id,actor_name
0,101,Robert Downey Jr.
1,102,Scarlett Johansson
2,103,Chris Hemsworth
3,104,Mark Ruffalo


In [7]:
# Q23 - Get the distinct film IDs from inventory table

# Close previous cursor and create a new one
cursor.close()
cursor = conn.cursor()

query = "SELECT DISTINCT film_id FROM inventory;"
fetch_and_display(query, title="Distinct Film IDs from Inventory")



=== Distinct Film IDs from Inventory ===


,film_id
0,1
1,2
2,3
3,4
4,5
...,...
953,996
954,997
955,998
956,999


# Functions

# Basic Aggregate Functions:

In [ ]:
# Question 1:

Retrieve the total number of rentals made in the Sakila database.

Hint: Use the COUNT() function.

In [13]:

query = "SELECT COUNT(*) AS total_rentals FROM rental;"
fetch_and_display(query, title="Total Number of Rentals")



=== Total Number of Rentals ===


,total_rentals
0,16044


In [ ]:
# Question 2:

Find the average rental duration (in days) of movies rented from the Sakila database.

Hint: Utilize the AVG() function.

In [15]:
query = """
SELECT AVG(rental_duration) AS average_rental_duration
FROM film;
"""
fetch_and_display(query, title="Average Rental Duration of Movies")


=== Average Rental Duration of Movies ===


,average_rental_duration
0,4.9850


# String Functions:

In [ ]:
# Question 3:

Display the first name and last name of customers in uppercase.

Hint: Use the UPPER () function.

In [18]:
query = """
SELECT UPPER(first_name) AS first_name_upper, 
       UPPER(last_name) AS last_name_upper
FROM customer;
"""
fetch_and_display(query, title="Customer Names in Uppercase")


=== Customer Names in Uppercase ===


,first_name_upper,last_name_upper
0,MARY,SMITH
1,PATRICIA,JOHNSON
2,LINDA,WILLIAMS
3,BARBARA,JONES
4,ELIZABETH,BROWN
...,...,...
594,TERRENCE,GUNDERSON
595,ENRIQUE,FORSYTHE
596,FREDDIE,DUGGAN
597,WADE,DELVALLE


In [ ]:
# Question 4:

Extract the month from the rental date and display it alongside the rental ID.

Hint: Employ the MONTH() function.

In [20]:
query = """
SELECT rental_id, MONTH(rental_date) AS rental_month
FROM rental;
"""
fetch_and_display(query, title="Rental ID with Month Extracted from Rental Date")



=== Rental ID with Month Extracted from Rental Date ===


,rental_id,rental_month
0,1,5
1,2,5
2,3,5
3,4,5
4,5,5
...,...,...
16039,13486,2
16040,15966,2
16041,11676,2
16042,14616,2


# GROUP BY:

In [ ]:
# Question 5:

Retrieve the count of rentals for each customer (display customer ID and the count of rentals).

Hint: Use COUNT () in conjunction with GROUP BY.

In [25]:
query = """
SELECT customer_id, COUNT(*) AS total_rentals
FROM rental
GROUP BY customer_id;
"""
fetch_and_display(query, title="Rental Count per Customer")


=== Rental Count per Customer ===


,customer_id,total_rentals
0,1,32
1,2,27
2,3,26
3,4,22
4,5,38
...,...,...
594,595,30
595,596,28
596,597,25
597,598,22


In [ ]:
# Question 6:

Find the total revenue generated by each store.

Hint: Combine SUM() and GROUP BY

In [33]:
query = """
SELECT i.store_id, SUM(p.amount) AS total_revenue
FROM payment p
JOIN rental r ON p.rental_id = r.rental_id
JOIN inventory i ON r.inventory_id = i.inventory_id
GROUP BY i.store_id;
"""
fetch_and_display(query, title="Total Revenue Generated by Each Store")



=== Total Revenue Generated by Each Store ===


,store_id,total_revenue
0,1,33679.79
1,2,33726.77


In [ ]:
# Question 7:

Determine the total number of rentals for each category of movies.

Hint: JOIN film_category, film, and rental tables, then use cOUNT () and GROUP BY.

In [35]:
query = """
SELECT fc.category_id, COUNT(r.rental_id) AS total_rentals
FROM rental r
JOIN inventory i ON r.inventory_id = i.inventory_id
JOIN film f ON i.film_id = f.film_id
JOIN film_category fc ON f.film_id = fc.film_id
GROUP BY fc.category_id;
"""
fetch_and_display(query, title="Total Rentals per Movie Category")


=== Total Rentals per Movie Category ===


,category_id,total_rentals
0,1,1112
1,2,1166
2,3,945
3,4,939
4,5,941
5,6,1050
6,7,1060
7,8,1096
8,9,1033
9,10,969


In [ ]:
# Question 8:

Find the average rental rate of movies in each language.

Hint: JOIN film and language tables, then use AVG () and GROUP BY.

In [37]:
query = """
SELECT l.name AS language, AVG(f.rental_rate) AS avg_rental_rate
FROM film f
JOIN language l ON f.language_id = l.language_id
GROUP BY l.language_id;
"""
fetch_and_display(query, title="Average Rental Rate per Language")


=== Average Rental Rate per Language ===


,language,avg_rental_rate
0,English,2.980000


# Joins

In [ ]:
# Questions 9 -

Display the title of the movie, customer s first name, and last name who rented it.

Hint: Use JOIN between the film, inventory, rental, and customer tables.

In [40]:
query = """
SELECT f.title AS movie_title, c.first_name, c.last_name
FROM film f
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
JOIN customer c ON r.customer_id = c.customer_id;
"""
fetch_and_display(query, title="Movies Rented with Customer Names")



=== Movies Rented with Customer Names ===


,movie_title,first_name,last_name
0,ACADEMY DINOSAUR,JOEL,FRANCISCO
1,ACADEMY DINOSAUR,GABRIEL,HARDER
2,ACADEMY DINOSAUR,DIANNE,SHELTON
3,ACADEMY DINOSAUR,NORMAN,CURRIER
4,ACADEMY DINOSAUR,BEATRICE,ARNOLD
...,...,...,...
16039,ZORRO ARK,JESSIE,BANKS
16040,ZORRO ARK,JACKIE,LYNCH
16041,ZORRO ARK,MAUREEN,LITTLE
16042,ZORRO ARK,TONY,CARRANZA


In [ ]:
# Question 10:

Retrieve the names of all actors who have appeared in the film "Gone with the Wind."

Hint: Use JOIN between the film actor, film, and actor tables.

In [42]:
query = """
SELECT a.first_name, a.last_name
FROM actor AS a
JOIN film_actor AS fa ON a.actor_id = fa.actor_id
JOIN film AS f ON fa.film_id = f.film_id
WHERE f.title = 'Gone with the Wind';
"""
fetch_and_display(query, title="Actors in 'Gone with the Wind'")


=== Actors in 'Gone with the Wind' ===


,first_name,last_name


In [ ]:
# Question 11:

Retrieve the customer names along with the total amount they've spent on rentals.

Hint: JOIN customer, payment, and rental tables, then use SUM() and GROUP BY.

In [44]:
query = """
SELECT c.first_name, c.last_name, SUM(p.amount) AS total_spent
FROM customer AS c
JOIN payment AS p ON c.customer_id = p.customer_id
JOIN rental AS r ON p.rental_id = r.rental_id
GROUP BY c.customer_id, c.first_name, c.last_name;
"""
fetch_and_display(query, title="Total Amount Spent by Each Customer")


=== Total Amount Spent by Each Customer ===


,first_name,last_name,total_spent
0,MARY,SMITH,118.68
1,PATRICIA,JOHNSON,128.73
2,LINDA,WILLIAMS,135.74
3,BARBARA,JONES,81.78
4,ELIZABETH,BROWN,144.62
...,...,...,...
594,TERRENCE,GUNDERSON,117.70
595,ENRIQUE,FORSYTHE,96.72
596,FREDDIE,DUGGAN,99.75
597,WADE,DELVALLE,83.78


In [ ]:
# Question 12:

List the titles of movies rented by each customer in a particular city (e.g., 'London').

Hint: JOIN customer, address, city, rental, inventory, and film tables, then use GROUP BY.

In [47]:
query = """
SELECT c.first_name, c.last_name, f.title
FROM customer AS c
JOIN address AS a ON c.address_id = a.address_id
JOIN city AS ci ON a.city_id = ci.city_id
JOIN rental AS r ON c.customer_id = r.customer_id
JOIN inventory AS i ON r.inventory_id = i.inventory_id
JOIN film AS f ON i.film_id = f.film_id
WHERE ci.city = 'London'
ORDER BY c.customer_id;
"""
fetch_and_display(query, title="Movies Rented by Customers in London")



=== Movies Rented by Customers in London ===


,first_name,last_name,title
0,MATTIE,HOFFMAN,CONQUERER NUTS
1,MATTIE,HOFFMAN,WRATH MILE
2,MATTIE,HOFFMAN,COLDBLOODED DARLING
3,MATTIE,HOFFMAN,DARKNESS WAR
4,MATTIE,HOFFMAN,WATERSHIP FRONTIER
5,MATTIE,HOFFMAN,JAWBREAKER BROOKLYN
6,MATTIE,HOFFMAN,TIGHTS DAWN
7,MATTIE,HOFFMAN,CHICKEN HELLFIGHTERS
8,MATTIE,HOFFMAN,DOOM DANCING
9,MATTIE,HOFFMAN,PITY BOUND


# Advanced Joins and GROUP BY:

In [ ]:
# Question 13:

Display the top 5 rented movies along with the number of times they've been rented.

Hint: JOIN film, inventory, and rental tables, then use COUNT () and GROUP BY, and limit the results.


In [49]:
query = """
SELECT f.title, COUNT(r.rental_id) AS rental_count
FROM film AS f
JOIN inventory AS i ON f.film_id = i.film_id
JOIN rental AS r ON i.inventory_id = r.inventory_id
GROUP BY f.film_id
ORDER BY rental_count DESC
LIMIT 5;
"""
fetch_and_display(query, title="Top 5 Rented Movies with Rental Count")


=== Top 5 Rented Movies with Rental Count ===


,title,rental_count
0,BUCKET BROTHERHOOD,34
1,ROCKETEER MOTHER,33
2,FORWARD TEMPLE,32
3,GRIT CLOCKWORK,32
4,JUGGLER HARDLY,32


In [ ]:
# Question 14:

Determine the customers who have rented movies from both stores (store ID 1 and store ID 2).

Hint: Use JOINS with rental, inventory, and customer tables and consider COUNT() and GROUP BY.

In [51]:
query = """
SELECT c.customer_id, c.first_name, c.last_name
FROM customer AS c
JOIN rental AS r ON c.customer_id = r.customer_id
JOIN inventory AS i ON r.inventory_id = i.inventory_id
WHERE i.store_id IN (1, 2)
GROUP BY c.customer_id
HAVING COUNT(DISTINCT i.store_id) = 2;
"""
fetch_and_display(query, title="Customers Renting from Both Stores 1 and 2")


=== Customers Renting from Both Stores 1 and 2 ===


,customer_id,first_name,last_name
0,1,MARY,SMITH
1,2,PATRICIA,JOHNSON
2,3,LINDA,WILLIAMS
3,4,BARBARA,JONES
4,5,ELIZABETH,BROWN
...,...,...,...
594,595,TERRENCE,GUNDERSON
595,596,ENRIQUE,FORSYTHE
596,597,FREDDIE,DUGGAN
597,598,WADE,DELVALLE


# Windows Function:

In [54]:
# 1. Rank the customers based on the total amount they've spent on rentals.

query = """
SELECT c.customer_id, c.first_name, c.last_name, SUM(p.amount) AS total_spent
FROM customer AS c
JOIN payment AS p ON c.customer_id = p.customer_id
GROUP BY c.customer_id
ORDER BY total_spent DESC;
"""
fetch_and_display(query, title="Customers Ranked by Total Amount Spent")


=== Customers Ranked by Total Amount Spent ===


,customer_id,first_name,last_name,total_spent
0,526,KARL,SEAL,221.55
1,148,ELEANOR,HUNT,216.54
2,144,CLARA,SHAW,195.58
3,137,RHONDA,KENNEDY,194.61
4,178,MARION,SNYDER,194.61
...,...,...,...,...
594,97,ANNIE,RUSSELL,58.82
595,395,JOHNNY,TURPIN,57.81
596,318,BRIAN,WYMAN,52.88
597,281,LEONA,OBRIEN,50.86


In [56]:
# 2. Calculate the cumulative revenue generated by each film over time.

query = """
SELECT f.film_id, f.title, r.rental_date, 
       SUM(p.amount) OVER (PARTITION BY f.film_id ORDER BY r.rental_date) AS cumulative_revenue
FROM film AS f
JOIN inventory AS i ON f.film_id = i.film_id
JOIN rental AS r ON i.inventory_id = r.inventory_id
JOIN payment AS p ON r.rental_id = p.rental_id
ORDER BY f.film_id, r.rental_date;
"""
fetch_and_display(query, title="Cumulative Revenue per Film Over Time")



=== Cumulative Revenue per Film Over Time ===


,film_id,title,rental_date,cumulative_revenue
0,1,ACADEMY DINOSAUR,2005-05-27 07:03:28,0.99
1,1,ACADEMY DINOSAUR,2005-05-30 20:21:07,2.98
2,1,ACADEMY DINOSAUR,2005-06-15 02:57:51,3.97
3,1,ACADEMY DINOSAUR,2005-06-17 20:24:00,4.96
4,1,ACADEMY DINOSAUR,2005-06-21 00:30:26,6.95
...,...,...,...,...
16039,1000,ZORRO ARK,2005-08-19 02:18:51,181.73
16040,1000,ZORRO ARK,2005-08-19 03:49:28,189.72
16041,1000,ZORRO ARK,2005-08-19 16:18:24,199.71
16042,1000,ZORRO ARK,2005-08-21 17:43:42,204.70


In [62]:
# 3. Determine the average rental duration for each film, considering films with similar lengths.

query = """
SELECT f.film_id, f.title, f.length, 
       AVG(f.rental_duration) AS avg_rental_duration
FROM film AS f
JOIN inventory AS i ON f.film_id = i.film_id
JOIN rental AS r ON i.inventory_id = r.inventory_id
GROUP BY f.film_id, f.title, f.length
ORDER BY f.length;
"""
fetch_and_display(query, title="Average Rental Duration per Film (Grouped by Length)")



=== Average Rental Duration per Film (Grouped by Length) ===


,film_id,title,length,avg_rental_duration
0,15,ALIEN CENTER,46,5.0000
1,469,IRON MOON,46,7.0000
2,504,KWAI HOMEWARD,46,5.0000
3,505,LABYRINTH LEAGUE,46,6.0000
4,730,RIDGEMONT SUBMARINE,46,3.0000
...,...,...,...,...
953,609,MUSCLE BRIGHT,185,7.0000
954,690,POND SEATTLE,185,7.0000
955,817,SOLDIERS EVOLUTION,185,7.0000
956,872,SWEET BROTHERHOOD,185,3.0000


In [64]:
# 4. Identify the top 3 films in each category based on their rental counts.

query = """
SELECT c.category_id, c.name AS category_name, f.film_id, f.title, COUNT(r.rental_id) AS rental_count
FROM film f
JOIN film_category fc ON f.film_id = fc.film_id
JOIN category c ON fc.category_id = c.category_id
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY c.category_id, f.film_id
ORDER BY c.category_id, rental_count DESC
LIMIT 3;
"""

fetch_and_display(query, title="Top 3 Films in Each Category by Rental Count")


=== Top 3 Films in Each Category by Rental Count ===


,category_id,category_name,film_id,title,rental_count
0,1,Action,748,RUGRATS SHAKESPEARE,30
1,1,Action,869,SUSPECTS QUILLS,30
2,1,Action,850,STORY SIDE,28


In [74]:
# 5. Calculate the difference in rental counts between each customer's total rentals and the average rentals across all customers.

query = """
SELECT c.customer_id, 
       COUNT(r.rental_id) AS customer_rentals,
       (COUNT(r.rental_id) - 
        (SELECT AVG(rental_count) 
         FROM (SELECT COUNT(rental_id) AS rental_count 
               FROM rental 
               GROUP BY customer_id) AS avg_table)
       ) AS diff_from_avg
FROM customer c
LEFT JOIN rental r ON c.customer_id = r.customer_id
GROUP BY c.customer_id;
"""

fetch_and_display(query, title="Difference from Average Rentals per Customer")


=== Difference from Average Rentals per Customer ===


,customer_id,customer_rentals,diff_from_avg
0,1,32,5.2154
1,2,27,0.2154
2,3,26,-0.7846
3,4,22,-4.7846
4,5,38,11.2154
...,...,...,...
594,595,30,3.2154
595,596,28,1.2154
596,597,25,-1.7846
597,598,22,-4.7846


In [76]:
# 6. Find the monthly revenue trend for the entire rental store over time.

query = """
SELECT DATE_FORMAT(payment_date, '%Y-%m') AS month, 
       SUM(amount) AS total_revenue
FROM payment
GROUP BY month
ORDER BY month;
"""

fetch_and_display(query, title="Monthly Revenue Trend")


=== Monthly Revenue Trend ===


,month,total_revenue
0,2005-05,4824.43
1,2005-06,9631.88
2,2005-07,28373.89
3,2005-08,24072.13
4,2006-02,514.18


In [84]:
# 7. Identify the customers whose total spending on rentals falls within the top 20% of all customers.

# Step 1: Calculate total spent per customer
query_total = """
SELECT customer_id, SUM(amount) AS total_spent
FROM payment
GROUP BY customer_id
ORDER BY total_spent DESC;
"""
cursor.execute(query_total)
total_results = cursor.fetchall()

# Step 2: Determine top 20% count
import pandas as pd
df_total = pd.DataFrame(total_results, columns=['customer_id', 'total_spent'])
top_20_count = int(len(df_total) * 0.2)

# Step 3: Get top 20% customers
df_top20 = df_total.head(top_20_count)
display(df_top20)


,customer_id,total_spent
0,526,221.55
1,148,216.54
2,144,195.58
3,137,194.61
4,178,194.61
...,...,...
114,473,134.66
115,433,133.75
116,59,133.73
117,92,133.72


In [86]:
# 8. Calculate the running total of rentals per category, ordered by rental count.

# Q8 - Calculate running total of rentals per category, ordered by rental count

# Step 1: Fetch rental counts per category
query = """
SELECT c.category_id, c.name AS category_name, COUNT(r.rental_id) AS rental_count
FROM category c
JOIN film_category fc ON c.category_id = fc.category_id
JOIN film f ON fc.film_id = f.film_id
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY c.category_id, c.name
ORDER BY rental_count DESC;
"""
cursor.execute(query)
result = cursor.fetchall()

# Step 2: Convert to DataFrame and calculate running total
import pandas as pd
df = pd.DataFrame(result, columns=['category_id', 'category_name', 'rental_count'])
df['running_total'] = df['rental_count'].cumsum()

# Step 3: Display result
display(df)


,category_id,category_name,rental_count,running_total
0,15,Sports,1179,1179
1,2,Animation,1166,2345
2,1,Action,1112,3457
3,14,Sci-Fi,1101,4558
4,8,Family,1096,5654
5,7,Drama,1060,6714
6,6,Documentary,1050,7764
7,9,Foreign,1033,8797
8,10,Games,969,9766
9,3,Children,945,10711


In [92]:
# 9. Find the films that have been rented less than the average rental count for their respective categories.

# Step 1: Fetch rental counts per film with category
query = """
SELECT f.film_id, f.title, c.category_id, c.name AS category_name, COUNT(r.rental_id) AS rental_count
FROM film f
JOIN film_category fc ON f.film_id = fc.film_id
JOIN category c ON fc.category_id = c.category_id
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY f.film_id, f.title, c.category_id, c.name;
"""
cursor.execute(query)
result = cursor.fetchall()

# Step 2: Convert to DataFrame
import pandas as pd
df = pd.DataFrame(result, columns=['film_id', 'title', 'category_id', 'category_name', 'rental_count'])

# Step 3: Calculate average rental count per category
category_avg = df.groupby('category_id')['rental_count'].transform('mean')

# Step 4: Filter films below average rental count
df_below_avg = df[df['rental_count'] < category_avg]

# Step 5: Display result
display(df_below_avg)


,film_id,title,category_id,category_name,rental_count
2,29,ANTITRUST TOMATOES,1,Action,10
3,56,BAREFOOT MANCHURIAN,1,Action,18
6,105,BULL SHAWSHANK,1,Action,16
7,111,CADDYSHACK JEDI,1,Action,16
9,126,CASUALTIES ENCINO,1,Action,9
...,...,...,...,...,...
950,894,TOMATOES HELLFIGHTERS,16,Travel,14
951,903,TRAFFIC HOBBIT,16,Travel,5
953,931,VALENTINE VANISHING,16,Travel,12
954,977,WINDOW SIDE,16,Travel,12


In [94]:
# 10. Identify the top 5 months with the highest revenue and display the revenue generated in each month.

# Step 1: Fetch payment data with month
query = """
SELECT MONTH(payment_date) AS month, SUM(amount) AS total_revenue
FROM payment
GROUP BY MONTH(payment_date)
ORDER BY total_revenue DESC
LIMIT 5;
"""
cursor.execute(query)
result = cursor.fetchall()

# Step 2: Convert to DataFrame
import pandas as pd
df = pd.DataFrame(result, columns=['month', 'total_revenue'])

# Step 3: Display the result
display(df)


,month,total_revenue
0,7,28373.89
1,8,24072.13
2,6,9631.88
3,5,4824.43
4,2,514.18


# Normalisation & CTE

# 1.First Normal Form (1NF):

 a. Identify a table in the Sakila database that violates 1NF. Explain how you

 would normalize it to achieve 1NF.

# Ans. 
a. Identify a table that violates 1NF
Table Chosen: movie
Assumed Structure in Mavenmovies:

CREATE TABLE movie (
    movie_id INT,
    title TEXT,
    genres TEXT  -- e.g., 'Action, Comedy'
);

 Why It Violates 1NF:
- The genres column contains multiple values in one cell (comma-separated).
- This breaks 1NF because:
- Values are not atomic.
- There are repeating groups in a single column.

How to Normalize It to Achieve 1NF
Split the data into two tables:

-- Table 1: movie
CREATE TABLE movie (
    movie_id INT PRIMARY KEY,
    title TEXT
);

-- Table 2: movie_genre
CREATE TABLE movie_genre (
    movie_id INT,
    genre TEXT
);

Now:
- Each genre is stored in a separate row.
- All values are atomic.
- This satisfies First Normal Form (1NF)


# 2. Second Normal Form (2NF):

 a. Choose a table in Sakila and describe how you would determine whether it is in 2NF. 

 If it violates 2NF, explain the step

# Ans:

choose a table and check if it violates 2NF
Table Chosen from Mavenmovies: movie_cast
Structure

CREATE TABLE movie_cast (
    movie_id INT,
    actor_id INT,
    actor_name TEXT,
    role TEXT,
    PRIMARY KEY (movie_id, actor_id)
);



 Why It Violates 2NF:
- The primary key is composite: (movie_id, actor_id)
- The column actor_name depends only on actor_id, not on the full key.
- This is a partial dependency, which violates 2NF.

 How to Normalize to 2NF
Split into two tables:

-- Table 1: actor
CREATE TABLE actor (
    actor_id INT PRIMARY KEY,
    actor_name TEXT
);

-- Table 2: movie_cast
CREATE TABLE movie_cast (
    movie_id INT,
    actor_id INT,
    role TEXT,
    PRIMARY KEY (movie_id, actor_id),
    FOREIGN KEY (actor_id) REFERENCES actor(actor_id)
);


Now:
- actor_name depends only on actor_id in the actor table.
- movie_cast contains only attributes that depend on the full composite key.
 This satisfies Second Normal Form (2NF).


# 3. Third Normal Form 3NF):

 a. Identify a table in Sakila that violates 3NF. Describe the transitive dependencies 

 present and outline the steps to normalize the table to 3NF.

# Ans:

Third Normal Form (3NF)

🔹a. Identify a table in Mavenmovies that violates 3NF

Table Chosen: customer

Assumed Structure:

CREATE TABLE customer (
    customer_id INT PRIMARY KEY,
    customer_name TEXT,
    city TEXT,
    state TEXT
);



 Transitive Dependency Present:

- customer_id → city

- city → state

This means:

- state is transitively dependent on customer_id via city.

- This violates 3NF because state is a non-key attribute depending on another non-key attribute (city), not directly on the primary key.

 How to Normalize to 3NF

Split into two tables:

-- Table 1: customer

CREATE TABLE customer (
    customer_id INT PRIMARY KEY,
    customer_name TEXT,
    city TEXT
);

-- Table 2: city_state

CREATE TABLE city_state (
    city TEXT PRIMARY KEY,
    state TEXT
);


Now:

- state depends directly on city in a separate table.

- All non-key attributes in customer depend only on the primary key (customer_id).

- This satisfies Third Normal Form (3NF).



# 4. Normalization Process:

 a. Take a specific table in Sakila and guide through the process of normalizing it from the initial 

 unnormalized form up to at least 2NF

# Ans: 

Table Chosen: film

🔹 Initial Unnormalized Form (UNF)

Assume the film table looks like this:

CREATE TABLE film (
    film_id INT,
    title TEXT,
    description TEXT,
    release_year INT,
    language TEXT,
    special_features TEXT
);


- special_features might contain multiple values like "Trailers, Deleted Scenes, Commentaries".

- language is stored as text, possibly repeated across many films.

🔹 Step 1: First Normal Form (1NF)

🔸 Goal: Eliminate repeating groups and ensure atomic values.
    
Issues:

- special_features contains comma-separated values — violates atomicity.

- language is a repeating textual value — not normalized.

🔹 Fix:

Split special_features into a separate table:

CREATE TABLE special_feature (
    feature_id INT PRIMARY KEY,
    feature_name TEXT
);

CREATE TABLE film_special_feature (
    film_id INT,
    feature_id INT,
    PRIMARY KEY (film_id, feature_id)
);


Also, normalize language:

CREATE TABLE language (
    language_id INT PRIMARY KEY,
    name TEXT
);

-- Update film table

CREATE TABLE film (
    film_id INT PRIMARY KEY,
    title TEXT,
    description TEXT,
    release_year INT,
    language_id INT,
    FOREIGN KEY (language_id) REFERENCES language(language_id)
);


Now the film table is in 1NF — all values are atomic and no repeating groups.

🔹 Step 2: Second Normal Form (2NF)

🔸 Goal: Eliminate partial dependencies on a composite key

In this case, film_id is a single-column primary key, so partial dependencies aren't an issue within film.

However, if we had a table like:

CREATE TABLE film_actor (
    film_id INT,
    actor_id INT,
    actor_name TEXT
);


Here, actor_name depends only on actor_id, not the full composite key (film_id, actor_id).

🔹 Fix:

Split actor_name into a separate actor table:

CREATE TABLE actor (
    actor_id INT PRIMARY KEY,
    actor_name TEXT
);

-- Update film_actor

CREATE TABLE film_actor (
    film_id INT,
    actor_id INT,
    PRIMARY KEY (film_id, actor_id),
    FOREIGN KEY (actor_id) REFERENCES actor(actor_id)
);


Now film_actor is in 2NF — all non-key attributes depend on the full key.

### Normalization Summary – Sakila Database

| Step       | Action Taken                                           | Result                     |
|------------|--------------------------------------------------------|----------------------------|
| UNF → 1NF  | Split multi-valued `special_features`, normalized `language` | Atomic values              |
| 1NF → 2NF  | Removed partial dependency in `film_actor`             | Full functional dependency |







In [ ]:
# 5. CTE Basics:

 a. Write a query using a CTE to retrieve the distinct list of actor names and the number of films they 

 have acted  in from the actor and film_actor tables.

In [34]:
# Ans: 

query = """
WITH ActorFilmCount AS (
    SELECT 
        a.actor_id,
        CONCAT(a.first_name, ' ', a.last_name) AS actor_name,
        COUNT(fa.film_id) AS film_count
    FROM actor a
    LEFT JOIN film_actor fa ON a.actor_id = fa.actor_id
    GROUP BY a.actor_id, a.first_name, a.last_name
)
SELECT actor_name, film_count
FROM ActorFilmCount
ORDER BY actor_name;
"""

fetch_and_display(query, "Distinct Actors and Number of Films")



=== Distinct Actors and Number of Films ===


,actor_name,film_count
0,ADAM GRANT,18
1,ADAM HOPPER,22
2,AL GARLAND,26
3,ALAN DREYFUSS,27
4,ALBERT JOHANSSON,33
...,...,...
195,WILL WILSON,31
196,WILLIAM HACKMAN,27
197,WOODY HOFFMAN,31
198,WOODY JOLIE,31


In [ ]:
# 6. CTE with Joins:

 a. Create a CTE that combines information from the film and language tables to display the film title, 

 language name, and rental rate.

In [36]:
# Ans:

query = """
WITH FilmLanguage AS (
    SELECT 
        f.film_id,
        f.title AS film_title,
        l.name AS language_name,
        f.rental_rate
    FROM film f
    JOIN language l ON f.language_id = l.language_id
)
SELECT film_title, language_name, rental_rate
FROM FilmLanguage
ORDER BY film_title;
"""

fetch_and_display(query, "Films with Language and Rental Rate")



=== Films with Language and Rental Rate ===


,film_title,language_name,rental_rate
0,ACADEMY DINOSAUR,English,0.99
1,ACE GOLDFINGER,English,4.99
2,ADAPTATION HOLES,English,2.99
3,AFFAIR PREJUDICE,English,2.99
4,AFRICAN EGG,English,2.99
...,...,...,...
995,YOUNG LANGUAGE,English,0.99
996,YOUTH KICK,English,0.99
997,ZHIVAGO CORE,English,0.99
998,ZOOLANDER FICTION,English,2.99


In [ ]:
# 7. CTE for Aggregation:

 a. Write a query using a CTE to find the total revenue generated by each customer (sum of payments) 

 from the customer and payment tables 

In [38]:
# Ans:

query = """
WITH CustomerRevenue AS (
    SELECT 
        c.customer_id,
        CONCAT(c.first_name, ' ', c.last_name) AS customer_name,
        SUM(p.amount) AS total_revenue
    FROM customer c
    JOIN payment p ON c.customer_id = p.customer_id
    GROUP BY c.customer_id, c.first_name, c.last_name
)
SELECT customer_name, total_revenue
FROM CustomerRevenue
ORDER BY total_revenue DESC;
"""

fetch_and_display(query, "Total Revenue per Customer")



=== Total Revenue per Customer ===


,customer_name,total_revenue
0,KARL SEAL,221.55
1,ELEANOR HUNT,216.54
2,CLARA SHAW,195.58
3,RHONDA KENNEDY,194.61
4,MARION SNYDER,194.61
...,...,...
594,ANNIE RUSSELL,58.82
595,JOHNNY TURPIN,57.81
596,BRIAN WYMAN,52.88
597,LEONA OBRIEN,50.86


In [ ]:
# 8. CTE with Window Functions:

 a. Utilize a CTE with a window function to rank films based on their rental duration from the film table.

In [40]:
# Ans:

query = """
WITH FilmRanking AS (
    SELECT 
        film_id,
        title AS film_title,
        rental_duration,
        RANK() OVER (ORDER BY rental_duration DESC) AS rental_rank
    FROM film
)
SELECT film_title, rental_duration, rental_rank
FROM FilmRanking
ORDER BY rental_rank;
"""

fetch_and_display(query, "Films Ranked by Rental Duration")



=== Films Ranked by Rental Duration ===


,film_title,rental_duration,rental_rank
0,ADAPTATION HOLES,7,1
1,ANONYMOUS HUMAN,7,1
2,ARGONAUTS TOWN,7,1
3,BIKINI BORROWERS,7,1
4,BLACKOUT PRIVATE,7,1
...,...,...,...
995,WISDOM WORKER,3,798
996,WONDERFUL DROP,3,798
997,WORDS HUNTER,3,798
998,WORLD LEATHERNECKS,3,798


In [ ]:
# 9. CTE and Filtering:

 a. Create a CTE to list customers who have made more than two rentals, and then join this CTE with the 

 customer table to retrieve additional customer details.

In [42]:
# Ans: 

query = """
WITH FrequentCustomers AS (
    SELECT 
        customer_id,
        COUNT(rental_id) AS rental_count
    FROM rental
    GROUP BY customer_id
    HAVING COUNT(rental_id) > 2
)
SELECT 
    c.customer_id,
    CONCAT(c.first_name, ' ', c.last_name) AS customer_name,
    c.email,
    fc.rental_count
FROM FrequentCustomers fc
JOIN customer c ON fc.customer_id = c.customer_id
ORDER BY fc.rental_count DESC;
"""

fetch_and_display(query, "Customers with More Than 2 Rentals")



=== Customers with More Than 2 Rentals ===


,customer_id,customer_name,email,rental_count
0,148,ELEANOR HUNT,ELEANOR.HUNT@sakilacustomer.org,46
1,526,KARL SEAL,KARL.SEAL@sakilacustomer.org,45
2,144,CLARA SHAW,CLARA.SHAW@sakilacustomer.org,42
3,236,MARCIA DEAN,MARCIA.DEAN@sakilacustomer.org,42
4,75,TAMMY SANDERS,TAMMY.SANDERS@sakilacustomer.org,41
...,...,...,...,...
594,248,CAROLINE BOWMAN,CAROLINE.BOWMAN@sakilacustomer.org,15
595,61,KATHERINE RIVERA,KATHERINE.RIVERA@sakilacustomer.org,14
596,110,TIFFANY JORDAN,TIFFANY.JORDAN@sakilacustomer.org,14
597,281,LEONA OBRIEN,LEONA.OBRIEN@sakilacustomer.org,14


In [ ]:
# 10. CTE for Date Calculations:

 a. Write a query using a CTE to find the total number of rentals made each month, considering the 

 rental_date from the rental table

In [44]:
# Ans:

query = """
WITH MonthlyRentals AS (
    SELECT 
        DATE_FORMAT(rental_date, '%Y-%m') AS rental_month,
        COUNT(rental_id) AS total_rentals
    FROM rental
    GROUP BY DATE_FORMAT(rental_date, '%Y-%m')
)
SELECT rental_month, total_rentals
FROM MonthlyRentals
ORDER BY rental_month;
"""

fetch_and_display(query, "Total Rentals per Month")



=== Total Rentals per Month ===


,rental_month,total_rentals
0,2005-05,1156
1,2005-06,2311
2,2005-07,6709
3,2005-08,5686
4,2006-02,182


In [ ]:
# 11.  CTE and Self-Join:

 a. Create a CTE to generate a report showing pairs of actors who have appeared in the same film 

 together, using the film_actor table.


In [46]:
# Ans: 

query = """
WITH ActorPairs AS (
    SELECT 
        fa1.film_id,
        fa1.actor_id AS actor1_id,
        fa2.actor_id AS actor2_id
    FROM film_actor fa1
    JOIN film_actor fa2 
        ON fa1.film_id = fa2.film_id 
       AND fa1.actor_id < fa2.actor_id
)
SELECT 
    CONCAT(a1.first_name, ' ', a1.last_name) AS actor1_name,
    CONCAT(a2.first_name, ' ', a2.last_name) AS actor2_name,
    ap.film_id
FROM ActorPairs ap
JOIN actor a1 ON ap.actor1_id = a1.actor_id
JOIN actor a2 ON ap.actor2_id = a2.actor_id
ORDER BY ap.film_id, actor1_name, actor2_name;
"""

fetch_and_display(query, "Actor Pairs in Same Film")



=== Actor Pairs in Same Film ===


,actor1_name,actor2_name,film_id
0,CHRISTIAN GABLE,JOHNNY CAGE,1
1,CHRISTIAN GABLE,LUCILLE TRACY,1
2,CHRISTIAN GABLE,MARY KEITEL,1
3,CHRISTIAN GABLE,MENA TEMPLE,1
4,CHRISTIAN GABLE,OPRAH KILMER,1
...,...,...,...
14910,PENELOPE CRONYN,WHOOPI HURT,999
14911,WHOOPI HURT,JADA RYDER,999
14912,IAN TANDY,LISA MONROE,1000
14913,IAN TANDY,NICK DEGENERES,1000


In [ ]:
# 12. CTE for Recursive Search:

 a. Implement a recursive CTE to find all employees in the staff table who report to a specific manager, 

 considering the reports_to column

In [54]:
# Step 1: Create the example employee table (only run once)
cursor.execute("""
CREATE TABLE IF NOT EXISTS employee (
    emp_id INT PRIMARY KEY,
    name VARCHAR(50),
    manager_id INT
);
""")

# Step 2: Insert sample data (only run once)
cursor.execute("""
INSERT INTO employee (emp_id, name, manager_id) VALUES
(1, 'Alice', NULL),
(2, 'Bob', 1),
(3, 'Charlie', 1),
(4, 'David', 2),
(5, 'Eve', 2),
(6, 'Frank', 3)
ON DUPLICATE KEY UPDATE name=VALUES(name);
""")

# Step 3: Recursive CTE query to find all employees reporting to manager_id = 1
manager_id = 1

query = f"""
WITH RECURSIVE StaffHierarchy AS (
    SELECT emp_id, name, manager_id
    FROM employee
    WHERE emp_id = {manager_id}

    UNION ALL

    SELECT e.emp_id, e.name, e.manager_id
    FROM employee e
    JOIN StaffHierarchy sh ON e.manager_id = sh.emp_id
)
SELECT emp_id, name, manager_id
FROM StaffHierarchy
WHERE emp_id != {manager_id}
ORDER BY manager_id, name;
"""

fetch_and_display(query, f"Employees Reporting to Manager ID {manager_id}")



=== Employees Reporting to Manager ID 1 ===


,emp_id,name,manager_id
0,2,Bob,1
1,3,Charlie,1
2,4,David,2
3,5,Eve,2
4,6,Frank,3


In [60]:
# After all queries
cursor.close()
conn.close()
